In [14]:
from utils.db_utils import gold_fragments_df_for_signer, get_labels_for_signer
import numpy as np
import pandas as pd


In [15]:

def flt2str(f): return '{:.2f}'.format(f)


def phn_vad_lines_for_seq(tmpdf, add_sil=True):
    phn_lines = []
    vad_lines = []
    
    lastend = 0
    for i,row in tmpdf.iterrows():
        fname, s,e,l = row.filename, row.start, row.end+1, row.labelname

        if (s != lastend) and (add_sil): # add silence first
            phn_lines.append(' '.join([fname,flt2str(lastend),flt2str(s),'SIL']))

        phn_lines.append(' '.join([fname,flt2str(s),flt2str(e),l]))
        vad_lines.append(' '.join([fname,flt2str(s),flt2str(e)]))
        
        lastend = e
    
    return phn_lines, vad_lines


def phn_vad_lines_for_gold_df(gold_df, add_sil=True):
    
    seq_names = sorted(gold_df.filename.unique())
    
    phn_lines = []
    vad_lines = []
    
    for fname in seq_names:
        tmpdf = gold_df.loc[gold_df.filename==fname].sort_values(by='start')
        phn, vad = phn_vad_lines_for_seq(tmpdf, add_sil)
        
        phn_lines.extend(phn)
        vad_lines.extend(vad)
            
    return phn_lines, vad_lines

In [16]:
def phn_vad_lines_for_signer(signer_id, group=3, add_sil=True, interp=False):

    gold_df = gold_fragments_df_for_signer(signer_id, group=group, interp=interp, excluded_labels=[])
    
    phn_lines, vad_lines = phn_vad_lines_for_gold_df(gold_df, add_sil=True)
    
    return phn_lines, vad_lines

In [13]:
i = 3
signer_id = 'Signer0' + str(i)

import os

def write2file(dest, name, lines): 
    with open(os.path.join(dest,name), 'w') as f: f.write('\n'.join(lines))

def write_tde_gold(signer_id):
    phn_lines, vad_lines = phn_vad_lines_for_signer(signer_id, group=3, add_sil=True)
    wrd_lines, _ = phn_vad_lines_for_signer(signer_id, group=3, add_sil=False, interp=True)

    write2file(dest, signer_id + '.phn', phn_lines)
    write2file(dest, signer_id + '.vad', vad_lines)
    write2file(dest, signer_id + '.wrd', wrd_lines)

    

In [17]:
dest

'/home/korhan/Desktop/tdev2-master/tdev2/share/Phoenix/'

In [18]:
for ext in ['phn','vad','wrd']:
    tmp = []
    for i in range(1,10):
        signer_id = 'Signer0' + str(i)
        with open(dest + signer_id + '.' + ext,'a') as f: f.write('\n')
            
    